In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

                                              0.0/165.7 kB ? eta -:--:--
     ----------------                        71.7/165.7 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 165.7/165.7 kB 2.0 MB/s eta 0:00:00
                                              0.0/7.8 MB ? eta -:--:--
     -                                        0.3/7.8 MB 7.7 MB/s eta 0:00:01
     ---                                      0.8/7.8 MB 9.6 MB/s eta 0:00:01
     ----------                               2.1/7.8 MB 17.0 MB/s eta 0:00:01
     -----------------------                  4.5/7.8 MB 28.5 MB/s eta 0:00:01
     ----------------------------             5.5/7.8 MB 24.9 MB/s eta 0:00:01
     ---------------------------------------  7.6/7.8 MB 28.5 MB/s eta 0:00:01
     ---------------------------------------  7.8/7.8 MB 27.5 MB/s eta 0:00:01
     ---------------------------------------- 7.8/7.8 MB 23.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build depend

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "C:\Users\User\Desktop\github\arasoft-langchain\myenv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\User\Desktop\github\arasoft-langchain\myenv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\User\Desktop\github\arasoft-langchain\myenv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\User\AppData\Local\Temp\pip-build-env-zlc1ti9o\overlay

^C


  Cloning https://github.com/unslothai/unsloth.git to c:\users\user\appdata\local\temp\pip-install-wuwqvjre\unsloth_fef6f8b05e6f4e6dae4e2c57d5fd6296
  Resolved https://github.com/unslothai/unsloth.git to commit 597e1894f60b9157490d99dc140714899bcc0d14
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
                                              0.0/105.9 kB ? eta -:--:--
     ---------------------------------       92.2/105.9 kB 5.1 MB/s eta 0:00:01
     -------------------------------------- 105.9/105.9 kB 6.0 MB/s eta 0:00:00
                                              0.0/9.5 MB ? eta -:--:--
                                              0.1/9.5 MB 2.4 MB/s eta 0:00:04
     --                             

  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git 'C:\Users\User\AppData\Local\Temp\pip-install-wuwqvjre\unsloth_fef6f8b05e6f4e6dae4e2c57d5fd6296'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
protoc-gen-openapiv2 0.0.1 requires protobuf>=4.21.0, but you have protobuf 3.20.3 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# xformers 버전 이슈 컨트롤
!pip uninstall xformers
!pip install xformers

In [5]:
# Base Model 세팅
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "sh2orc/Llama-3.1-Korean-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

ModuleNotFoundError: No module named 'bitsandbytes'

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from datasets import load_dataset

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    # 책 제목, 대분류, 중분류를 하나의 문자열로 결합하여 instruction에 할당
    instructions = [
        f"책 제목: {title} 대분류: {category1} 중분류: {category2}"
        for title, category1, category2 in zip(examples["책_제목"], examples["대분류_카테고리"], examples["중분류_카테고리"])
    ]
    # 원고를 input에 할당
    inputs = examples["원고_텍스트"]
    # HTML을 output에 할당
    outputs = examples["HTML"]

    # 최종 텍스트 포맷팅
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts, }
pass

# 예시 데이터셋 전처리 적용
base_path = '/content/gdrive/MyDrive/아라소프트/'
csv_file_path = base_path + 'dataset.csv'
dataset = load_dataset('csv', data_files=csv_file_path, split = "train")
dataset = dataset.map(formatting_prompts_func, batched=True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")